# `torch.jit`

Eager execution is great for development and debugging. but it can be hard to (automatically) optimize the code and deploy it.

Now there is`torch.jit` with two flavours:

- `torch.jit.trace` does not record control flow.
- `torch.jit.script` records control flow and creates an intermediate representation that can be optimized; only supports a subset of Python.

Note: don't forget `model.eval()` and `model.train()`.


## Ref and More:
- https://pytorch.org/docs/stable/jit.html
- https://speakerdeck.com/perone/pytorch-under-the-hood
- https://lernapparat.de/fast-lstm-pytorch/

## Init, helpers, utils, ...

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

In [ ]:
from pprint import pprint

import matplotlib.pyplot as plt
import numpy as np
from IPython.core.debugger import set_trace

import utils  # little helpers
from utils import attr

# `torch.jit.trace`

In [ ]:
def f(x):
    if x.item() < 0:
        return torch.tensor(0)
    else:
        return x

In [ ]:
f(torch.tensor(-1))

In [ ]:
f(torch.tensor(3))

In [ ]:
X = torch.tensor(1)
traced = torch.jit.trace(f, X)

In [ ]:
type(traced)

In [ ]:
traced(torch.tensor(1))

In [ ]:
traced.graph

In [ ]:
traced(torch.tensor(-1))

## Storing and restoring

In [ ]:
traced.save("traced.pt")

In [ ]:
!file scripted.pt

In [ ]:
g = torch.jit.load("traced.pt")

In [ ]:
g(torch.tensor(1))

In [ ]:
g(torch.tensor(-1))

# `torch.jit.script`

In [ ]:
bool(torch.tensor(1) < 2)

In [ ]:
@torch.jit.script
def f(x):
    if bool(x < 0):
        result = torch.zeros(1)
    else:
        result = x
    return result

This is `torchscript` which is a only a supset of python.

In [ ]:
f(torch.tensor(-1))

In [ ]:
f(torch.tensor(1))

In [ ]:
type(f)

In [ ]:
f.graph

## Storing and restoring

In [ ]:
torch.jit.save(f, "scripted.pt")

In [ ]:
!file scripted.pt

In [ ]:
g = torch.jit.load("scripted.pt")

In [ ]:
g(torch.tensor(-1))

In [ ]:
g(torch.tensor(1))

## Subclassing `torch.jit.ScriptModule`
If you work with `nn.Module` replace it by `torch.jit.ScriptModule` (see [[tutorial]](https://pytorch.org/tutorials/beginner/deploy_seq2seq_hybrid_frontend_tutorial.html) for more).

```python
class MyModule(torch.jit.ScriptModule):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        # ...
        return x
```

# PyTorch and C++

PyTorch offers a very nice(!) C++ interface which is very close to Python.

## Loading traced models from C++

```c++
#include <torch/script.h>

int main(int(argc, const char* argv[]) {
    auto module = torch::jit::load("scrpted.pt");
    // data ...
    module->forward(data);
}
```